In [ ]:
import pandas as pd
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.1-8B-Instruct", legacy=False, device_map = "cuda")
PAD_TOKEN = "<|pad|>"
tokenizer.add_special_tokens({"pad_token": PAD_TOKEN})
tokenizer.padding_side = "right"

In [ ]:
df = pd.read_csv("training_df.csv")
df = df[df['gold_standard_label'].notna()]

def format_example(row):
    messages = [
        {
            "role": "system",
            "content": "Answer questions to the best of your ability."
        },
        {
            "role": "user",
            "content": row['training_prompt']
        },
        {
            "role": "assistant",
            "content": str(int(row['gold_standard_label']))
        }
    ]
    return tokenizer.apply_chat_template(messages, tokenize=False)

df['text'] = df.apply(format_example, axis=1)

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=17)
train_df.to_json("train.json", orient='records', lines=True)
test_df.to_json("holdout_test.json", orient='records', lines=True)